In [ ]:
pip install transformers

Importing all the libraries that will be needed later

In [ ]:
import os

import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.layers import Dense, Input
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import ModelCheckpoint

import transformers
from tqdm.notebook import tqdm
from tokenizers import BertWordPieceTokenizer

Encode Function

In [ ]:
def fast_encode(texts, tokenizer, chunk_size=256, maxlen=512):
   
    tokenizer.enable_truncation(max_length=maxlen)
    tokenizer.enable_padding()
    all_ids = []
    
    for i in tqdm(range(0, len(texts), chunk_size)):
        text_chunk = texts[i:i+chunk_size].tolist()
        encs = tokenizer.encode_batch(text_chunk)
        all_ids.extend([enc.ids for enc in encs])
    
    return np.array(all_ids)

Build  model function

In [ ]:
def build_model(transformer, max_len=512):
    
    input_word_ids = Input(shape=(max_len,), dtype=tf.int32, name="input_word_ids")
    sequence_output = transformer(input_word_ids)[0]
    cls_token = sequence_output[:, 0, :]
    out = Dense(1, activation='sigmoid')(cls_token)
    
    model = Model(inputs=input_word_ids, outputs=out)
    model.compile(Adam(lr=1e-5), loss='binary_crossentropy', metrics=['accuracy'])
    
    return model

In [ ]:
# Detect hardware, return appropriate distribution strategy
try:
    # TPU detection. No parameters necessary if TPU_NAME environment variable is
    # set: this is always the case on Kaggle.
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
    print('Running on TPU ', tpu.master())
except ValueError:
    tpu = None

if tpu:
    tf.config.experimental_connect_to_cluster(tpu)
    tf.tpu.experimental.initialize_tpu_system(tpu)
    strategy = tf.distribute.experimental.TPUStrategy(tpu)
else:
    # Default distribution strategy in Tensorflow. Works on CPU and single GPU.
    strategy = tf.distribute.get_strategy()

print("REPLICAS: ", strategy.num_replicas_in_sync)

Running on TPU  grpc://10.101.62.250:8470
INFO:tensorflow:Deallocate tpu buffers before initializing tpu system.


INFO:tensorflow:Deallocate tpu buffers before initializing tpu system.


INFO:tensorflow:Initializing the TPU system: grpc://10.101.62.250:8470


INFO:tensorflow:Initializing the TPU system: grpc://10.101.62.250:8470


INFO:tensorflow:Finished initializing TPU system.


...

INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


REPLICAS:  8


In [ ]:
AUTO = tf.data.experimental.AUTOTUNE
EPOCHS = 3
BATCH_SIZE = 16 * strategy.num_replicas_in_sync
MAX_LEN = 192

Usinf Distilbert Base(Multilingual cased)

In [ ]:
tokenizer = transformers.DistilBertTokenizer.from_pretrained('distilbert-base-multilingual-cased')
tokenizer.save_pretrained('.')
fast_tokenizer = BertWordPieceTokenizer('vocab.txt', lowercase=False)
fast_tokenizer

Tokenizer(vocabulary_size=119547, model=BertWordPiece, unk_token=[UNK], sep_token=[SEP], cls_token=[CLS], pad_token=[PAD], mask_token=[MASK], clean_text=True, handle_chinese_chars=True, strip_accents=None, lowercase=False, wordpieces_prefix=##)

In [ ]:
train1 = pd.read_csv("/content/drive/MyDrive/NLPMANDATE/jigsaw-toxic-comment-train.csv")
train2 = pd.read_csv("/content/drive/MyDrive/NLPMANDATE/jigsaw-unintended-bias-train.csv")
train2.toxic = train2.toxic.round().astype(int)

valid = pd.read_csv('/content/drive/MyDrive/NLPMANDATE/validation.csv')
test = pd.read_csv('/content/drive/MyDrive/NLPMANDATE/test.csv')

Forming the train data from train1 and train2

In [ ]:
train = pd.concat([
    train1[['comment_text', 'toxic']],
    train2[['comment_text', 'toxic']].query('toxic==1'),
    train2[['comment_text', 'toxic']].query('toxic==0').sample(n=150000, random_state=0)
])

In [ ]:
test

,id,comment_text
0,00001cee341fdb12,Yo bitch Ja Rule is more succesful then you'll...
1,0000247867823ef7,== From RfC == \n\n The title is fine as it is...
2,00013b17ad220c46,""" \n\n == Sources == \n\n * Zawe Ashton on Lap..."
3,00017563c3f7919a,":If you have a look back at the source, the in..."
4,00017695ad8997eb,I don't anonymously edit articles at all.
...,...,...
153159,fffcd0960ee309b5,". \n i totally agree, this stuff is nothing bu..."
153160,fffd7a9a6eb32c16,== Throw from out field to home plate. == \n\n...
153161,fffda9e8d6fafa9e,""" \n\n == Okinotorishima categories == \n\n I ..."
153162,fffe8f1340a79fc2,""" \n\n == """"One of the founding nations of the..."


In [ ]:
train

,comment_text,toxic
0,Explanation\nWhy the edits made under my usern...,0
1,D'aww! He matches this background colour I'm s...,0
2,"Hey man, I'm really not trying to edit war. It...",0
3,"""\nMore\nI can't make any real suggestions on ...",0
4,"You, sir, are my hero. Any chance you remember...",0
...,...,...
824002,"I watched her whole speech and she was witty, ...",0
688368,“Ms. Huffington says she does not feel that co...,0
1008737,"Rather than letting the hypocritical, madhouse...",0
864308,"The US. economy is ""ferocious""? Keep drinking...",0


Using the encode function here on "comment_text" and tokenizer being fast_tokenizer

In [ ]:

x_valid = fast_encode(valid.comment_text.astype(str), fast_tokenizer, maxlen=MAX_LEN)
x_test = fast_encode(test.comment_text.astype(str), fast_tokenizer, maxlen=MAX_LEN)

y_train = train.toxic.values
y_valid = valid.toxic.values

  0%|          | 0/32 [00:00<?, ?it/s]

  0%|          | 0/599 [00:00<?, ?it/s]

In [ ]:
x_train = fast_encode(train.comment_text.astype(str), fast_tokenizer,maxlen=MAX_LEN)

  0%|          | 0/1898 [00:00<?, ?it/s]

In [ ]:
x_train

array([[   101,  27746,  31609, ...,      0,      0,      0],
       [   101,    141,    112, ...,      0,      0,      0],
       [   101,  35936,  10817, ...,      0,      0,      0],
       ...,
       [   101, 104390,  11084, ...,  13409,  16091,    102],
       [   101,  10117,  10808, ...,      0,      0,      0],
       [   101,  11038,  14985, ...,      0,      0,      0]])

In [ ]:
y_train

array([0, 0, 0, ..., 0, 0, 0])

Converting into tensors


In [ ]:
train_dataset = (
    tf.data.Dataset
    .from_tensor_slices((x_train, y_train))
    .repeat()
    .shuffle(2048)
    .batch(BATCH_SIZE)
    .prefetch(AUTO)
)

valid_dataset = (
    tf.data.Dataset
    .from_tensor_slices((x_valid, y_valid))
    .batch(BATCH_SIZE)
    .cache()
    .prefetch(AUTO)
)

test_dataset = (
    tf.data.Dataset
    .from_tensor_slices(x_test)
    .batch(BATCH_SIZE)
)

In [ ]:
train_dataset

<PrefetchDataset element_spec=(TensorSpec(shape=(None, 192), dtype=tf.int64, name=None), TensorSpec(shape=(None,), dtype=tf.int64, name=None))>

Model

In [ ]:
%%time
with strategy.scope():
    transformer_layer = (
        transformers.TFDistilBertModel
        .from_pretrained('distilbert-base-multilingual-cased')
    )
    model = build_model(transformer_layer, max_len=MAX_LEN)
model.summary()

Some layers from the model checkpoint at distilbert-base-multilingual-cased were not used when initializing TFDistilBertModel: ['vocab_projector', 'vocab_layer_norm', 'vocab_transform', 'activation_13']
- This IS expected if you are initializing TFDistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFDistilBertModel were initialized from the model checkpoint at distilbert-base-multilingual-cased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFDistilBertModel for predictions without further training.


Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_word_ids (InputLayer)  [(None, 192)]            0         
                                                                 
 tf_distil_bert_model_1 (TFD  TFBaseModelOutput(last_h  134734080
 istilBertModel)             idden_state=(None, 192,             
                             768),                               
                              hidden_states=None, att            
                             entions=None)                       
                                                                 
 tf.__operators__.getitem_1   (None, 768)              0         
 (SlicingOpLambda)                                               
                                                                 
 dense_1 (Dense)             (None, 1)                 769       
                                                           

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


In [ ]:
 n_steps = x_train.shape[0]


In [ ]:
n_steps=n_steps

In [ ]:
n_steps

485775

Training

In [ ]:
n_steps = x_train.shape[0] // BATCH_SIZE
train_history = model.fit(
    train_dataset,
    steps_per_epoch=n_steps,
    validation_data=valid_dataset,
    epochs=EPOCHS
)

Epoch 1/3
3795/3795 [==============================] - 464s 122ms/step - loss: 0.0620 - accuracy: 0.9754 - val_loss: 0.3591 - val_accuracy: 0.8499
Epoch 2/3
3795/3795 [==============================] - 451s 119ms/step - loss: 0.0500 - accuracy: 0.9800 - val_loss: 0.4865 - val_accuracy: 0.8478
Epoch 3/3
3795/3795 [==============================] - 450s 119ms/step - loss: 0.0424 - accuracy: 0.9830 - val_loss: 0.5317 - val_accuracy: 0.8480


In [ ]:
n_steps = x_valid.shape[0] // BATCH_SIZE


In [ ]:
n_steps

62

Training on Validation dataset

In [ ]:
n_steps = x_valid.shape[0] // BATCH_SIZE
train_history_2 = model.fit(
    valid_dataset.repeat(),
    steps_per_epoch=n_steps,
    epochs=EPOCHS*2
)

62/62 [==============================] - 7s 118ms/step - loss: 0.2438 - accuracy: 0.8942


In [ ]:
sub = pd.read_csv('/content/drive/MyDrive/NLPMANDATE/sample_submission.csv')

Predicting on the Text Data

In [ ]:
sub['toxic'] = model.predict(test_dataset, verbose=1)

1197/1197 [==============================] - 52s 41ms/step


In [ ]:
sub

,id,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,00001cee341fdb12,0.954891,0.5,0.5,0.5,0.5,0.5
1,0000247867823ef7,0.006472,0.5,0.5,0.5,0.5,0.5
2,00013b17ad220c46,0.044195,0.5,0.5,0.5,0.5,0.5
3,00017563c3f7919a,0.003224,0.5,0.5,0.5,0.5,0.5
4,00017695ad8997eb,0.040085,0.5,0.5,0.5,0.5,0.5
...,...,...,...,...,...,...,...
153159,fffcd0960ee309b5,0.043714,0.5,0.5,0.5,0.5,0.5
153160,fffd7a9a6eb32c16,0.138652,0.5,0.5,0.5,0.5,0.5
153161,fffda9e8d6fafa9e,0.022427,0.5,0.5,0.5,0.5,0.5
153162,fffe8f1340a79fc2,0.144785,0.5,0.5,0.5,0.5,0.5


In [ ]:
test['toxic']=sub['toxic']

In [ ]:
test

,id,comment_text,toxic
0,00001cee341fdb12,Yo bitch Ja Rule is more succesful then you'll...,0.954891
1,0000247867823ef7,== From RfC == \n\n The title is fine as it is...,0.006472
2,00013b17ad220c46,""" \n\n == Sources == \n\n * Zawe Ashton on Lap...",0.044195
3,00017563c3f7919a,":If you have a look back at the source, the in...",0.003224
4,00017695ad8997eb,I don't anonymously edit articles at all.,0.040085
...,...,...,...
153159,fffcd0960ee309b5,". \n i totally agree, this stuff is nothing bu...",0.043714
153160,fffd7a9a6eb32c16,== Throw from out field to home plate. == \n\n...,0.138652
153161,fffda9e8d6fafa9e,""" \n\n == Okinotorishima categories == \n\n I ...",0.022427
153162,fffe8f1340a79fc2,""" \n\n == """"One of the founding nations of the...",0.144785


Test data and the corresponding level of toxicity(between 0 and 1)

In [ ]:
test.drop(['id'], axis = 1)

,comment_text,toxic
0,Yo bitch Ja Rule is more succesful then you'll...,0.954891
1,== From RfC == \n\n The title is fine as it is...,0.006472
2,""" \n\n == Sources == \n\n * Zawe Ashton on Lap...",0.044195
3,":If you have a look back at the source, the in...",0.003224
4,I don't anonymously edit articles at all.,0.040085
...,...,...
153159,". \n i totally agree, this stuff is nothing bu...",0.043714
153160,== Throw from out field to home plate. == \n\n...,0.138652
153161,""" \n\n == Okinotorishima categories == \n\n I ...",0.022427
153162,""" \n\n == """"One of the founding nations of the...",0.144785


In [ ]:
test['comment_text'][0]

"Yo bitch Ja Rule is more succesful then you'll ever be whats up with you and hating you sad mofuckas...i should bitch slap ur pethedic white faces and get you to kiss my ass you guys sicken me. Ja rule is about pride in da music man. dont diss that shit on him. and nothin is wrong bein like tupac he was a brother too...fuckin white boys get things right next time.,"

,id,comment_text,lang,toxic
0,0,Este usuario ni siquiera llega al rango de ...,es,0
1,1,Il testo di questa voce pare esser scopiazzato...,it,0
2,2,Vale. Sólo expongo mi pasado. Todo tiempo pasa...,es,1
3,3,Bu maddenin alt başlığı olarak uluslararası i...,tr,0
4,4,Belçika nın şehirlerinin yanında ilçe ve belde...,tr,0
...,...,...,...,...
7995,7995,Il fatto è che la pagina dei personaggi minor...,it,0
7996,7996,El imbesil ete dela luna no se entera ni ostia...,es,1
7997,7997,olum sız manyakmısınz siz adam sıze sanal yıld...,tr,1
7998,7998,El mapa del reinado de Alhaken esta ligerament...,es,0
